# Project 2


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  #To get figures with high quality!

import numpy as np
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import time
import torchvision

## Loading the dataset CIFAR10

We are told to classify birds from cats in the CIFAR10 dataset. This means that we will only use the images that are labeled as birds or cats. In the CIFAR10 these labels are 2 and 3 respectively. We will have the training, validation and test datasets.

In [153]:
from torchvision import datasets, transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#Filtering the birds and cats images from the train set
trainset.targets = torch.tensor(trainset.targets)
mask = (trainset.targets == 2) | (trainset.targets == 3)
trainset.targets = (trainset.targets[mask] - 2)  # Adjust labels to be 0 for the birds and 1 cats
trainset.data = trainset.data[mask.numpy().astype(bool)]
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)


testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#Filtering the birds and cats images from the test set
testset.targets = torch.tensor(testset.targets)
mask = (testset.targets == 2) | (testset.targets == 3)
testset.targets = (testset.targets[mask] - 2)  # Adjust labels to be 0 for the birds and 1 cats
testset.data = testset.data[mask.numpy().astype(bool)]

testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


Splitting the training into train and valid (80/20)

100 25
100 25


## Implementation of a Lenet5

In [157]:
class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_part = nn.Sequential( # As images are rgb we have 3 input channels
            nn.Conv2d(in_channels=3, out_channels=6,
                      kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120), # The input dimension at the classifier is 16 images of 5x5 so 16*5*5
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 2), # As we are just classifying two classes the output must be 2
            nn.LogSigmoid()

        )
        self.lr = 0.001 #Learning Rate

        self.optim = optim.Adam(self.parameters(), self.lr)

        self.criterion = nn.CrossEntropyLoss()

        self.loss_during_training = []

        self.valid_loss_during_training = []


    def forward(self, x):
        x = self.conv_part(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def trainloop(self,trainloader,validloader,epochs):

        self.train()
        for e in range(epochs):

            running_loss = 0.

            for images, labels in trainloader:

                self.optim.zero_grad()

                out = self.forward(images)


                loss = self.criterion(out,labels)

                running_loss += loss.item()


                loss.backward()
                self.optim.step()

            self.loss_during_training.append(running_loss/len(trainloader))

            with torch.no_grad():

                  self.eval()

                  running_loss = 0.

                  for images,labels in validloader:


                      out = self.forward(images)

                      #Loss function
                      loss = self.criterion(out,labels)

                      running_loss += loss.item()

                  self.valid_loss_during_training.append(running_loss/len(validloader))

                  self.train()


            print("Epoch", e  ,". Training loss: ",self.loss_during_training[-1]," Valid loss: ",self.valid_loss_during_training[-1])

    def compute_accuracy(self, dataloader):
      correct = 0
      total = 0
      with torch.no_grad():
        self.eval()
        for data in dataloader:
                images, labels = data
                outputs = self(images)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
      return accuracy


In [158]:
model = Lenet5()
model.trainloop(trainloader,testloader,30)

Epoch 0 . Training loss:  0.5934792548228222  Valid loss:  0.5239438144490123
Epoch 1 . Training loss:  0.48836401655415823  Valid loss:  0.4764425475150347
Epoch 2 . Training loss:  0.45911102120284064  Valid loss:  0.4716479741036892
Epoch 3 . Training loss:  0.43069018708292844  Valid loss:  0.46636530943214893
Epoch 4 . Training loss:  0.41172183556541514  Valid loss:  0.4406569553539157
Epoch 5 . Training loss:  0.39494435489177704  Valid loss:  0.44568847957998514
Epoch 6 . Training loss:  0.37349235612875337  Valid loss:  0.4377274885773659
Epoch 7 . Training loss:  0.3515718026905303  Valid loss:  0.4339816989377141
Epoch 8 . Training loss:  0.33702642845500047  Valid loss:  0.45053990837186575
Epoch 9 . Training loss:  0.31685814128559864  Valid loss:  0.4725383296608925
Epoch 10 . Training loss:  0.3024973916779658  Valid loss:  0.44063428696244955
Epoch 11 . Training loss:  0.2824957162901095  Valid loss:  0.48118692077696323
Epoch 12 . Training loss:  0.2615491388140211  Va

In [159]:
print(model.compute_accuracy(testloader))

0.793
